In [3]:
# Notebook 03 – Modélisation 3 (temps + coût de stock)
# Cellule 1 — Imports
# But : importer les librairies nécessaires (NumPy pour les calculs, random pour le hasard).

import numpy as np
import random


In [5]:
#On conserve les durées des actions (P1 = 1, P2 = 3, commande = 1, attendre = 1).
#On ajoute un coût de stockage à chaque step

# Cellule 2 — Classe WorkshopEnvV3 (temps + coût de stockage)

class WorkshopEnvV3:
    """
    Environnement d'atelier avec :
    - durée variable des actions
    - coût de stockage sur les produits finis (stock_sell)
    """

    def __init__(self, max_steps=50, holding_cost_per_unit=0.5):
        self.max_steps = max_steps
        self.holding_cost_per_unit = holding_cost_per_unit
        self.reset()

    def reset(self):
        # Stock de matière initial aléatoire entre 0 et 10
        self.stock_raw = np.random.randint(0, 11)
        # Aucun produit fini au départ
        self.stock_sell = 0

        # Compteur de temps
        self.steps = 0

        return (self.stock_raw, self.stock_sell)

    def step(self, action):
        reward = 0.0

        # Durées des actions
        duration_P1 = 1
        duration_P2 = 3
        duration_cmd = 1
        duration_wait = 1

        # ACTION 1 : Produire Produit 1 (rapide, faible marge)
        if action == 1:
            if self.stock_raw >= 1:
                self.stock_raw -= 1
                self.stock_sell += 1
                reward = 2.0
            # durée
            self.steps += duration_P1

        # ACTION 2 : Produire Produit 2 (lent, forte marge)
        elif action == 2:
            if self.stock_raw >= 2:
                self.stock_raw -= 2
                self.stock_sell += 1
                reward = 20.0
            # durée
            self.steps += duration_P2

        # ACTION 3 : Commander du stock
        elif action == 3:
            self.stock_raw += 5
            reward = -5.0
            self.steps += duration_cmd

        # ACTION 0 : Attendre
        elif action == 0:
            # si on n'a plus de matière première, attendre est pénalisé
            reward = -1.0 if self.stock_raw == 0 else 0.0
            self.steps += duration_wait

        # BORNES sur les stocks
        self.stock_raw = max(0, min(10, self.stock_raw))
        self.stock_sell = max(0, min(10, self.stock_sell))

        # ➜ COÛT DE STOCKAGE : pénalise les gros stocks de produits finis
        # plus il y a de produits en stock, plus le coût par step est élevé
        reward -= self.holding_cost_per_unit * self.stock_sell

        # FIN D'ÉPISODE
        done = self.steps >= self.max_steps

        next_state = (self.stock_raw, self.stock_sell)
        return next_state, reward, done, {}




In [7]:
# Cellule 3 — Instanciation de l'environnement V3

env = WorkshopEnvV3(max_steps=50, holding_cost_per_unit=0.5)


In [9]:
# Cellule 4 — Table-Q et hyperparamètres

n_states_raw = 11       # 0 à 10
n_states_sell = 11      # 0 à 10
n_actions = 4           # 0,1,2,3

Q = np.zeros((n_states_raw, n_states_sell, n_actions))

alpha = 0.1     # taux d'apprentissage
gamma = 0.95    # facteur d'actualisation

epsilon = 1.0
epsilon_min = 0.1
epsilon_decay = 0.995


In [11]:
# Cellule 5 — Choix d'action ε-greedy

def choose_action(state, epsilon):
    stock_raw, stock_sell = state

    # Exploration
    if random.random() < epsilon:
        return random.randint(0, n_actions - 1)

    # Exploitation
    return int(np.argmax(Q[stock_raw, stock_sell, :]))


In [13]:
# Cellule 6 — Boucle d'entraînement (Q-learning)

n_episodes = 5000
rewards_per_episode = []

for episode in range(n_episodes):
    state = env.reset()
    done = False
    total_reward = 0.0

    while not done:
        # 1. Choix de l'action
        action = choose_action(state, epsilon)

        # 2. Transition environnement
        next_state, reward, done, info = env.step(action)

        # 3. Indices
        sr, ss = state
        nsr, nss = next_state

        # 4. Mise à jour Q (formule de Bellman)
        best_next_Q = np.max(Q[nsr, nss, :])
        Q[sr, ss, action] += alpha * (reward + gamma * best_next_Q - Q[sr, ss, action])

        # 5. Passage à l'état suivant
        state = next_state
        total_reward += reward

    rewards_per_episode.append(total_reward)

    # 6. Décroissance de epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    # Affichage périodique
    if (episode + 1) % 100 == 0:
        print(f"Episode {episode+1} : Reward total = {total_reward:.1f}, epsilon = {epsilon:.3f}")


Episode 100 : Reward total = 80.5, epsilon = 0.606
Episode 200 : Reward total = 147.0, epsilon = 0.367
Episode 300 : Reward total = 175.5, epsilon = 0.222
Episode 400 : Reward total = 171.0, epsilon = 0.135
Episode 500 : Reward total = 192.5, epsilon = 0.100
Episode 600 : Reward total = 196.5, epsilon = 0.100
Episode 700 : Reward total = 170.5, epsilon = 0.100
Episode 800 : Reward total = 178.5, epsilon = 0.100
Episode 900 : Reward total = 176.5, epsilon = 0.100
Episode 1000 : Reward total = 172.0, epsilon = 0.100
Episode 1100 : Reward total = 148.5, epsilon = 0.100
Episode 1200 : Reward total = 167.5, epsilon = 0.100
Episode 1300 : Reward total = 173.0, epsilon = 0.100
Episode 1400 : Reward total = 178.5, epsilon = 0.100
Episode 1500 : Reward total = 202.0, epsilon = 0.100
Episode 1600 : Reward total = 192.5, epsilon = 0.100
Episode 1700 : Reward total = 169.5, epsilon = 0.100
Episode 1800 : Reward total = 198.5, epsilon = 0.100
Episode 1900 : Reward total = 195.5, epsilon = 0.100
Epi

In [15]:
# Cellule 7 — Inspection de la politique apprise
# On garde la vue stock_sell = 0
# On ajoute une vue pour un stock_sell = 5, pour voir l’impact du coût de stockage.

def best_action_for_state(stock_raw, stock_sell):
    q_vals = Q[stock_raw, stock_sell, :]
    best_a = int(np.argmax(q_vals))
    return best_a, q_vals

print("=== Politique pour stock_sell = 0 ===")
for sr in range(0, 11):
    action, qv = best_action_for_state(sr, 0)
    print(f"État ({sr},0) → meilleure action = {action}, Q-values = {qv}")

print("\n=== Politique pour stock_sell = 5 ===")
for sr in range(0, 11):
    action, qv = best_action_for_state(sr, 5)
    print(f"État ({sr},5) → meilleure action = {action}, Q-values = {qv}")


=== Politique pour stock_sell = 0 ===
État (0,0) → meilleure action = 3, Q-values = [133.31860218  94.98779372  79.77205835 175.15136115]
État (1,0) → meilleure action = 3, Q-values = [104.99000358 116.83207795 112.35871144 177.97601061]
État (2,0) → meilleure action = 2, Q-values = [ 99.31144431 127.89621109 158.18469774  97.58495668]
État (3,0) → meilleure action = 2, Q-values = [102.34886997  30.65284206 174.42969773 119.6138362 ]
État (4,0) → meilleure action = 2, Q-values = [126.57934962 123.27095736 175.7049548  119.45689886]
État (5,0) → meilleure action = 2, Q-values = [165.3983305  158.47766891 189.63301175 172.43468284]
État (6,0) → meilleure action = 2, Q-values = [148.50730943 128.15169943 192.60632702 160.04474642]
État (7,0) → meilleure action = 2, Q-values = [ 99.05122397 131.36797499 170.98389237 125.82492145]
État (8,0) → meilleure action = 2, Q-values = [113.04229084 118.18601041 188.08389239 124.20166209]
État (9,0) → meilleure action = 2, Q-values = [ 90.14812288 13

In [17]:
# Cellule — Politique complète, version métier

actions = {
    0: "Attendre",
    1: "Produire P1",
    2: "Produire P2",
    3: "Commander"
}

print("=== Politique métier complète ===\n")

for ss in range(0, 11):
    print(f"## Stock de produits finis = {ss}\n")
    for sr in range(0, 11):
        best_a = int(np.argmax(Q[sr, ss, :]))
        qv = Q[sr, ss, :]
        print(f"État stock_raw={sr}, stock_sell={ss} → {actions[best_a]} (Q = {qv})")
    print("\n")


=== Politique métier complète ===

## Stock de produits finis = 0

État stock_raw=0, stock_sell=0 → Commander (Q = [133.31860218  94.98779372  79.77205835 175.15136115])
État stock_raw=1, stock_sell=0 → Commander (Q = [104.99000358 116.83207795 112.35871144 177.97601061])
État stock_raw=2, stock_sell=0 → Produire P2 (Q = [ 99.31144431 127.89621109 158.18469774  97.58495668])
État stock_raw=3, stock_sell=0 → Produire P2 (Q = [102.34886997  30.65284206 174.42969773 119.6138362 ])
État stock_raw=4, stock_sell=0 → Produire P2 (Q = [126.57934962 123.27095736 175.7049548  119.45689886])
État stock_raw=5, stock_sell=0 → Produire P2 (Q = [165.3983305  158.47766891 189.63301175 172.43468284])
État stock_raw=6, stock_sell=0 → Produire P2 (Q = [148.50730943 128.15169943 192.60632702 160.04474642])
État stock_raw=7, stock_sell=0 → Produire P2 (Q = [ 99.05122397 131.36797499 170.98389237 125.82492145])
État stock_raw=8, stock_sell=0 → Produire P2 (Q = [113.04229084 118.18601041 188.08389239 124.201